In [1]:
import pandas as pd
import numpy as np
from math import log2
data = {'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain', 'Rain', 'Overcast', 'Sunny', 'Sunny', 'Rain', 'Sunny', 'Overcast', 'Overcast', 'Rain'],
    'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot', 'Mild'],
    'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'High'],
    'Windy': [False, True, False, False, False, True, True, False, False, False, True, True, False, True],
    'PlayTennis': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']}
df = pd.DataFrame(data)
def entropy(target_column):
    elements, counts = np.unique(target_column, return_counts=True)
    entropy_val = np.sum([(-counts[i] / np.sum(counts)) * np.log2(counts[i] / np.sum(counts)) for i in range(len(elements))])
    return entropy_val
def information_gain(data, split_attribute_name, target_name="PlayTennis"):
    total_entropy = entropy(data[target_name])
    values, counts = np.unique(data[split_attribute_name], return_counts=True)
    weighted_entropy = np.sum(
        [(counts[i] / np.sum(counts)) * entropy(data.where(data[split_attribute_name] == values[i]).dropna()[target_name]) for i in range(len(values))])
    information_gain_val = total_entropy - weighted_entropy
    return information_gain_val
def id3(data, original_data, features, target_attribute_name="PlayTennis", parent_node_class=None):
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    elif len(data) == 0:
        return np.unique(original_data[target_attribute_name])[
            np.argmax(np.unique(original_data[target_attribute_name], return_counts=True)[1])]
    best_feature = max(features, key=lambda x: information_gain(data, x, target_attribute_name))

    tree = {best_feature: {}}
    features = [i for i in features if i != best_feature]
    for value in np.unique(data[best_feature]):
        value = value
        sub_data = data.where(data[best_feature] == value).dropna()
        subtree = id3(sub_data, data, features, target_attribute_name, parent_node_class)
        tree[best_feature][value] = subtree

    return tree
features = df.columns[:-1].tolist()
tree = id3(df, df, features)
import pprint
pprint.pprint(tree)


{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Windy': {False: 'Yes', True: 'No'}},
             'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}
